# Finding Light

In this notebook we show how to find the stylespace channel for light.

First, we create a stylegan for bedrooms and show a few generated images.

In [ ]:
import torch
from rewrite_utils import zdataset, show, labwidget, paintwidget, renormalize, nethook, imgviz, pbar
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image
from rewrite_utils.stylegan2 import load_seq_stylegan
#import clip


gan_model = load_seq_stylegan('bedroom', mconv='seq', truncation=0.90)
nethook.set_requires_grad(False, gan_model)
zds = zdataset.z_dataset_for_model(gan_model, size=1000)

image_numbers = range(100)#  [46] [77] [59] [95]
def pilim(idata):
    return renormalize.as_image(idata)
show([
    [pilim(gan_model(zds[i][0][None].cuda())[0]), f'Image {i}']
    for i in image_numbers])

Notice that image number 41 has really nice light patterns cast from the lamps.

Let's find out how the model represents those light patterns.

First step is, we will manually make a mask over the light patterns we're interested in:

In [ ]:
image_number = 41

PW = paintwidget.PaintWidget(image=renormalize.as_url(
    gan_model(zds[image_number][0][None].cuda())[0]))
def do_reset():
    PW.mask = ''
RB = labwidget.Button('reset').on('click', do_reset)
show([PW, RB])

# I'll pre-draw a mask I drew for the experiment.
PW.mask = 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAQAAAAEACAYAAABccqhmAAAMfElEQVR4Xu3dv48cZxkH8HcWI3FpsJsonosUR7RInEsQ2EtPgUUBZST+AKCjAlNRQnokiwIJKjv/wVxAtLZEQ0cs5dZWhGRHQo4U2zdozmd0Nvdjf8yz+/jez3WRd5593s/z3jc7e7M7TfFDgEC1Ak21K7dwAgSKALAJCFQsIAAqHr6lExAA9gCBigUEQMXDt3QCAsAeIFCxgACoePiWTkAA2AMEKhYQABUP39IJCAB7gEDFAgKg4uFbOgEBYA8QqFhAAFQ8fEsnIADsAQIVCwiAiodv6QQEgD1AoGIBAVDx8C2dgACwBwhULCAAKh6+pRMQAPYAgYoFBEDFw7d0AgLAHiBQsYAAqHj4lk5AANgDBCoWEAAVD9/SCQgAe4BAxQICoOLhWzoBAWAPEKhYQABUPHxLJyAA7AECFQsIgIqHb+kEBIA9QKBiAQFQ8fAtnYAAsAcIVCwgACoevqUTEAD2AIGKBQRAxcO3dAICwB4gULGAAKh4+JZOQADYAwQqFhAAFQ/f0gkIAHuAQMUCAqDi4Vs6AQFgDxCoWEAAVDx8SycgAOwBAhULCICKh2/pBASAPUCgYgEBUPHwLZ2AALAHCFQsIAAqHr6lExAA9gCBigUEQMXDt3QCAsAeIFCxgACoePiWTkAA2AMEKhYQABUP39IJCIAF9sCjR9OLly51jxc4xEMJpBYQAKeMZzab/rzvy7dKKTtNU3YOH/pJKeWTpildKeWPly93w3/7IfBGCgiAY8b22WfTnadPy60jv/THDrfvy+OmKTfbtvvwjZy+pqsXEACvbYG9venNpim/XnBndFtb5cZYpwdONRbU9/ClBQTAEbrhJX8p5XdLanZt231/mWOdaiyj5pgxBATAoeKDB9Mr+/vlbtOUiyvA/qJtu9/Pe7xTjXmlPC5KQAAcys5m0+FNveurQA/vCUwm5eo8bwxmONVYZa2OPR8CAqCUMvyf+NmzcneMkfZ9+c32dnfztFqbOtUYY31qnC8BAVBKefBg+kHfl1sjjXa3bbvpSbU2caox0rqUOYcCAqCUMptNh/P2n40x3+E0YHu7u3RSrXWfaoyxJjXOr4AAeBEAK5//H90iTVPeP+59gHWfapzfbWtlYwkIgIAAaNvuWNd1nmqMtUHUOd8CAqCUsuQ78iftjPtt21057h/Xeapxvret1Y0lIABevAL4YSnl9kioH7VtN9T7v591nWqMtA5lKhAQAC/+CnCl78u/Rpr3iRcDjR0AJ51qjLQOZSoQEACHQx7p5fn9ra2yc9JnAtZ1qlHBvrXEkQQEwCHk8AGcJ09K1zQHH/9d6ufChXL17be7eycdvK5TjaWad1CVAgLgyNhX+TPdPFcArutUo8qdbNFLCQiA19gOQ+BOKeW9OUU/L+XgOwHm+hDQOk415uzbwwgUAXDMJhhOB774ogzX8591deBu05QP5vnwz8unWcephn1NYF4BAXCK1OGrgeEvBMNXgr28vn/4CrB7TVPuXb7cDVcQLvwTfaqxcEMOqFZAAGxo9NGnGhtalqd9wwQEwAYHFnmqscFleeo3SEAAJBhW1KlGgqVpIbmAAEg+IO0RiBQQAJG6ahNILiAAkg9IewQiBQRApK7aBJILCIDkA9IegUgBARCpu0LtR4++896lS3+/v0IJhxI4U0AAnEm0ngfs7V2/XUrzzVL67aZptg6f9Vnfl/9MJuVu35dftW33t/V041lqERAAG570w4fXfvz8eXPryC/9CR31/fCtRW27+6MNt+zpz5GAANjgMD/9dNpNJovdjWj42vG33vpyx+nBBgd3jp5aAGxomMNL/qZpjv3uwLNaOuveA2cd798JvBQQAKfshajbdM9m0++W0n9cSrO0f9/3d7a3d2/YygRWEVh6A67ypFmPXddtumez6eNSytdXcxjeE2iueWNwNcXajxYAhzcHffq03GqasnPahhheejfNwbf/fLjsxhne9Nvfn/x52eOPHre/X3bffffk+xCO8RxqnG+B6gNgyW/q7ba2yo2Tvv33tC0zm03/UEr56Ujb6vO27S6OVEuZCgWqDoBN3KZ7b+/63aZpTn2lMf8+7Pu23Z3M/3iPJPCqQLUBsKnbdI9z/v/KEL/nfQC/1ssKVBsAY9ylZ3hPYDIpVxf5UtCxA8DdgZbd+o4bBKoMgFW+lPP1bTPP/QCOHrPMxT+nbNXnbdtdsJUJLCtQZQBs8jbde3vXfts0k18uO7BXj+sftu3u5XFqqVKjQJUBMNLNOQ72y6JX5b24CKj8dYzN5mKgMRTrrlFrAAzf5399rNE3TXl/kfcBRvpLwPOtrS+/4TMBY02xzjoCYIS5L/pG3PBZ/ydPvvrPpmm+tuzTTyb7P3nnnY//suzxjiNQ7ZuAS178c9KOud+23ZVFt9MqVwS6AnBRbY8/SaDWVwDDp/Buj7QtPmrbbqlP9R2GwJ9KKV+Zp5f+4CsBig8BzYPlMXMJVBkAmW7TfXg6cGeOqwOHuxD/wEU/c+1rD5pToMoAGGxG+kvA/a2tsrPMZwJen8+LbwYqO30/+fZk8r8PJf277/t/NE2zO+/tx+ecu4cROBCoNgDcpttvAIGKA2AY/ipXBC56BaDNRiCjQLWvAF4Ow226M25LPa1LoPoAGKDdpntd283zZBMQAEcm4jbd2banfqIFBEC0sPoEEgsIgMTD0RqBaAEBEC2sPoHEAgIg8XC0RiBaQABEC6tPILGAAEg8HK0RiBYQANHC6hNILCAAEg9HawSiBQRAtLD6BBILCIDEw9EagWgBARAtrD6BxAICIPFwtEYgWkAARAurTyCxgABIPBytEYgWEADRwuoTSCwgABIPR2sEogUEQLSw+gQSCwiAxMPRGoFoAQEQLaw+gcQCAiDxcLRGIFpAAEQLq08gsYAASDwcrRGIFhAA0cLqE0gsIAASD0drBKIFBEC0sPoEEgsIgMTD0RqBaAEBEC2sPoHEAgIg8XC0RiBaQABEC6tPILGAAEg8HK0RiBYQANHC6hNILCAAEg9HawSiBQRAtLD6BBILCIDEw9EagWgBARAtrD6BxAICIPFwtEYgWkAARAurTyCxgABIPBytEYgWEADRwuoTSCwgABIPR2sEogUEQLSw+gQSCwiAxMPRGoFoAQEQLaw+gcQCAiDxcLRGIFpAAEQLq08gsYAASDwcrRGIFhAA0cLqE0gsIAASD0drBKIFBEC0sPoEEgsIgMTD0RqBaAEBEC2sPoHEAgIg8XC0RiBaQABEC6tPILGAAEg8HK0RiBYQANHC6hNILCAAEg9HawSiBQRAtLD6BBILCIDEw9EagWgBARAtrD6BxAICIPFwtEYgWkAARAurTyCxgABIPBytEYgWEADRwuoTSCwgABIPR2sEogUEQLSw+gQSCwiAxMPRGoFoAQEQLaw+gcQCAiDxcLRGIFpAAEQLq08gsYAASDwcrRGIFhAA0cLqE0gsIAASD0drBKIFBEC0sPoEEgsIgMTD0RqBaAEBEC2sPoHEAgIg8XC0RiBaQABEC6tPILGAAEg8HK0RiBYQANHC6hNILCAAEg9HawSiBQRAtLD6BBILCIDEw9EagWgBARAtrD6BxAICIPFwtEYgWkAARAurTyCxgABIPBytEYgWEADRwuoTSCwgABIPR2sEogUEQLSw+gQSCwiAxMPRGoFoAQEQLaw+gcQCAiDxcLRGIFpAAEQLq08gsYAASDwcrRGIFhAA0cLqE0gsIAASD0drBKIFBEC0sPoEEgsIgMTD0RqBaAEBEC2sPoHEAgIg8XC0RiBaQABEC6tPILGAAEg8HK0RiBYQANHC6hNILCAAEg9HawSiBQRAtLD6BBILCIDEw9EagWgBARAtrD6BxAICIPFwtEYgWkAARAurTyCxgABIPBytEYgWEADRwuoTSCwgABIPR2sEogUEQLSw+gQSCwiAxMPRGoFoAQEQLaw+gcQCAiDxcLRGIFpAAEQLq08gsYAASDwcrRGIFhAA0cLqE0gsIAASD0drBKIFBEC0sPoEEgsIgMTD0RqBaAEBEC2sPoHEAgIg8XC0RiBaQABEC6tPILGAAEg8HK0RiBYQANHC6hNILCAAEg9HawSiBQRAtLD6BBILCIDEw9EagWgBARAtrD6BxAICIPFwtEYgWkAARAurTyCxgABIPBytEYgWEADRwuoTSCwgABIPR2sEogUEQLSw+gQSCwiAxMPRGoFoAQEQLaw+gcQCAiDxcLRGIFpAAEQLq08gsYAASDwcrRGIFhAA0cLqE0gs8F99KF8fqMSP9QAAAABJRU5ErkJggg=='

In [ ]:
PW.image = PW.mask

In [ ]:
#PW.image

Now we can hunt through all the layers and identify S channels that, when zeroed, alter the masked pixels the most.  We will go through many layers and units.

In [ ]:
device = 'cuda'
mask = renormalize.from_url(PW.mask, target='pt')[0].to(device)
z = zds[image_number][0][None].cuda()
test_result = []
for layername in pbar([f'layer{l}' for l in range(3, 14)]): # or 3, 14 for all the layers
    with nethook.Trace(gan_model, f'{layername}.sconv.mconv.modulation') as slayer:
        baseline = gan_model(z)[0]
    for u in range(slayer.output.style.shape[1]):
        def edit_style(output):
            output.style[:, u] = 0
            return output
        with nethook.Trace(gan_model, f'{layername}.sconv.mconv.modulation', edit_output=edit_style):
            changed = gan_model(z)[0]
            delta = ((baseline - changed).abs() * mask).sum()
            test_result.append((delta.item(), (layername, u)))


Sort through the tested units and visualize the best 12.

In [ ]:
top = list(reversed(sorted(test_result)))
out = []
for rank, (score, (layername, u)) in enumerate(top[:20]):
    with nethook.Trace(gan_model, f'{layername}.sconv.mconv.modulation', edit_output=edit_style):
        changed = gan_model(zds[image_number][0][None].cuda())[0]
        delta = (baseline - changed)
        out.append([pilim(delta).resize((180, 180)), f'#{rank+1}: {layername} neuron {u}'])
show(out)

 * layer8 265 looks like it switches lights cleanly, both lampshade and reflectiion
 * layer11 398 controls reflected light only, without lampshade
 * layer11 112 controls yellowness of reflected light
 * layer9 252 specifically controls umbra shadows from light
 


Let's make a simple widget to control one of these.

In [ ]:
from importlib import reload

reload(labwidget)
rw = labwidget.Range()
im = labwidget.Image()
cp = labwidget.ColorPicker('#ffffff', desc='color: ')
num = labwidget.Textbox(desc='imgnum: ')
lay = labwidget.Textbox(desc='layername: ')
uni = labwidget.Textbox(desc='unitnum: ')

num.value = image_number
lay.value = 'layer8'
uni.value = 265


def get_lit_scene(imgnum, amount, layername, unitnum):
    def change_light(output):
        output.style[:, int(unitnum)] = 10 * amount
        return output
    with nethook.Trace(gan_model, f'{layername}.sconv.mconv.modulation', edit_output=change_light):
        return gan_model(zds[imgnum][0][None].cuda())[0]

baseline = get_lit_scene(int(num.value), -1.0, lay.value, uni.value)
im.render(pilim(baseline))

def readcolor():
    try:
        floatcolor = [float(int(cp.value[i:i+2], 16))/255.0  for i in [1,3,5]]
        color = torch.tensor(floatcolor).float()
        if len(color) == 3:
            return color
    except:
        pass
    return torch.tensor([1.0, 1.0, 1.0]).float()

def newimage():
    frac = ((float(rw.value) * 2 - 100) / 100.0)
    lit = get_lit_scene(int(num.value), frac, lay.value, uni.value)
    baseline = get_lit_scene(int(num.value), -1.0, lay.value, uni.value)
    light = lit - baseline
    color = readcolor().cuda()[:,None,None]
    colored = baseline + (light * color)
    im.render(pilim(colored))

show([[num, '', lay, '', uni, '', rw, im]])

num.on('value', newimage)
cp.on('value', newimage)
rw.on('value', newimage)
None